In [ ]:
pip install langchain chromadb pypdf gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [35]:
import gradio as gr
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

# Optimized PDF loading (first 5 pages)
def load_pdf(pdf_path, num_pages=5):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages[:num_pages]):
            text += page.extract_text() + "\n"
    return text

# Optimized summarization
def summarize_pdf(pdf_file):
    document_text = load_pdf(pdf_file.name)

    # Split text into smaller chunks for faster processing
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.create_documents([document_text])

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
    vector_store = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")
   # llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_type="mistral", device="cuda")

    llm = CTransformers( model="distilgpt2", model_type="gpt2", device="cuda")
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type="stuff")

    # Generate the summary
    summary = qa_chain.run("Summarize the document in simple terms.")
    print(summary)

    return summary

# Gradio Interface
iface = gr.Interface(fn=summarize_pdf,
                     inputs=gr.File(label="Upload PDF"),
                     outputs=gr.Textbox(label="Summary"))

# Launch with share=True
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://58e7fbcf8aee413238.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
